---
title: "26-因果推断中的协变量匹配"
author: "Simonzhou"
date: "2025-08-02"
date-modified: "2025-08-02"
format: 
    html:
        code-fold: false
        code-line-numbers: true
        code-highlight: true
        fig_caption: true
        number-sections: true
        toc: true
        toc-depth: 3
---

In [9]:
import stata_setup
stata_setup.config('C:/Program Files/Stata18', 'mp', splash=False)

这里将继续使用 `cfps2010.dta` 这个数据演示上述命令的使用方法。对该数据基本情况的介绍请参见第四章第五节。

与第四章演示线性回归时相同，本章分析的因变量仍为 `lninc`，干预变量为 `college`。

一元线性回归结果显示，在不控制任何变量的情况下，`college` 对 `lninc` 的回归系数为 0.824，这意味着，考上大学的样本和没有考上大学的样本在 `lninc` 上的均值差为 0.824。不过，考虑到这两个子样本在 `hukou`、`age`、`gender`、`race`、`sib-ling` 和 `fmedu` 这几个协变量上存在很大差异，上述结果很可能是有偏的。

下面，我们将通过匹配法来校正干预组和控制组在这些协变量上的差异。

In [10]:
%%stata
use "../../Data/causal-inference/cfps2010.dta", clear
reg lninc college,vce(cluster provcd)


. use "../../Data/causal-inference/cfps2010.dta", clear

. reg lninc college,vce(cluster provcd)

Linear regression                               Number of obs     =      4,137
                                                F(1, 24)          =     271.17
                                                Prob > F          =     0.0000
                                                R-squared         =     0.1095
                                                Root MSE          =     1.1498

                                (Std. err. adjusted for 25 clusters in provcd)
------------------------------------------------------------------------------
             |               Robust
       lninc | Coefficient  std. err.      t    P>|t|     [95% conf. interval]
-------------+----------------------------------------------------------------
     college |    .823612   .0500155    16.47   0.000     .7203851     .926839
       _cons |   9.353189   .1084703    86.23   0.000     9.129317    9.57

我们使用的第一种方法是精确匹配。首先，我们仅对 hukou 这一个变量实施精确匹配，具体如下：

In [11]:
%%stata
teffects nnmatch (lninc) (college),ematch(hukou)


Treatment-effects estimation                   Number of obs      =      4,137
Estimator      : nearest-neighbor matching     Matches: requested =          1
Outcome model  : matching                                     min =        786
Distance metric: Mahalanobis                                  max =       1594
------------------------------------------------------------------------------
             |              AI robust
       lninc | Coefficient  std. err.      z    P>|z|     [95% conf. interval]
-------------+----------------------------------------------------------------
ATE          |
     college |
 (是 vs 否)  |   .8021249   .0337823    23.74   0.000     .7359127     .868337
------------------------------------------------------------------------------


可以发现，在对 hukou 实施了精确匹配之后，college 对 lninc 的平均干预效应下降到了 0.802。

如果我们同时对 hukou、age、gender、race、sibling 和 fmedu 实施精确匹配，其结果又会如何呢？

In [12]:
%%stata
teffects nnmatch (lninc) (college),ematch(hukou age gender race sibling fmedu)

SystemError: no exact matches for observation 4; use option osample() to identify all
observations with deficient matches
r(459);


从结果的输出看，Stata 给出了一个错误提示，提示内容无法为所有个案找到在所有协变量上取值都相同的匹配对象。

于此同时，Stata 还给出了一个建议，即使用 option `osample()` 去标记那些匹配失败的个案。

参照这个建议，我们在增加选项 `osample()` 后重新执行了上述命令。输出结果显示，有 593 名个案无法按照我们设定的要求实施精确匹配。

对标识变量 `overlap` 进行统计描述也可以发现，有 593 名个案违反了共同取值范围假定。

In [13]:
%%stata
teffects nnmatch (lninc) (college),ematch(hukou age gender race sibling fmedu) osample(overlap)

SystemError: 593 observations have no exact matches; they are identified in the osample()
variable
r(459);


In [14]:
%%stata
tab overlap


    overlap |
  violation |
  indicator |      Freq.     Percent        Cum.
------------+-----------------------------------
          0 |      3,544       85.67       85.67
          1 |        593       14.33      100.00
------------+-----------------------------------
      Total |      4,137      100.00
